In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from gensim.models import doc2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn import preprocessing

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import plot_model

In [ ]:
import spacy

In [ ]:
from tqdm import tqdm
import random

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
#!pip install contextualSpellCheck

In [ ]:
import contextualSpellCheck

In [ ]:
#!pip install autocorrect

In [ ]:
from autocorrect import Speller

In [ ]:
from spacy.matcher import Matcher

In [ ]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, anneal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Fake reviews detection/chi_hotel_dataset.csv')

In [ ]:
#df = pd.read_csv('restaurant_reviews_anonymized.csv', encoding='ISO-8859-1')

In [ ]:
df.head()

,Real,Review
0,1,Let me begin by saying that there are two kind...
1,1,The only place inside the Loop that you can st...
2,1,I have walked by the Tokyo Hotel countless tim...
3,1,"If you are considering staying here, watch thi..."
4,1,"This place is disgusting, absolutely horrible,..."


In [ ]:
#df.rename(columns = {'Reviews':'Review'}, inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5854 entries, 0 to 5853
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Real    5854 non-null   int64 
 1   Review  5854 non-null   object
dtypes: int64(1), object(1)
memory usage: 91.6+ KB


In [ ]:
#df = df[~df['Review'].isnull()]

In [ ]:
#df.reset_index(inplace=True)

## Data preprocessing

In [ ]:
new_data = []

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
for line in df['Review']:
    a = re.sub('[^a-zA-Z]', ' ', line).lower()
    word_tokens = word_tokenize(a)
    filtered = [w for w in word_tokens if not w in stop_words]
    new_data.append(' '.join(filtered))

In [ ]:
new_data[:5]

['let begin saying two kinds people give tokyo hotel stars rave everyone know people get past broken phone blood stains beeping fire alarms peg legged receptionist lack water pressure cracked walls strange smells questionable elevator televisions left die digital conversion possibility air conditioner may fall window moment said whole heartedly give tokyo hotel stars place quietly slip nothing show faint memory imitation thomas kinkade painting bolted wall bed continental breakfast coffee lobby vending machines pretty sure wont take change minted senses assaulted leave enough memories compete mile road trip beg anyone even mildly considering staying give chance location prime able walk michigan ave river walk middle night without straying far hotel grocery store block away parking may cost hotel room across street besides place cheap super cheap downtown chicago closest price found area four times expensive sure grab cash accept credit cards rules though say hello clifton jackson homel

In [ ]:
final_data = [x for x in new_data if x != '']

In [ ]:
tagged_data = [TaggedDocument(words=word_tokenize(_d), tags=[str(i)]) for i, _d in enumerate(final_data)]

In [ ]:
tagged_data[:2]

[TaggedDocument(words=['let', 'begin', 'saying', 'two', 'kinds', 'people', 'give', 'tokyo', 'hotel', 'stars', 'rave', 'everyone', 'know', 'people', 'get', 'past', 'broken', 'phone', 'blood', 'stains', 'beeping', 'fire', 'alarms', 'peg', 'legged', 'receptionist', 'lack', 'water', 'pressure', 'cracked', 'walls', 'strange', 'smells', 'questionable', 'elevator', 'televisions', 'left', 'die', 'digital', 'conversion', 'possibility', 'air', 'conditioner', 'may', 'fall', 'window', 'moment', 'said', 'whole', 'heartedly', 'give', 'tokyo', 'hotel', 'stars', 'place', 'quietly', 'slip', 'nothing', 'show', 'faint', 'memory', 'imitation', 'thomas', 'kinkade', 'painting', 'bolted', 'wall', 'bed', 'continental', 'breakfast', 'coffee', 'lobby', 'vending', 'machines', 'pretty', 'sure', 'wont', 'take', 'change', 'minted', 'senses', 'assaulted', 'leave', 'enough', 'memories', 'compete', 'mile', 'road', 'trip', 'beg', 'anyone', 'even', 'mildly', 'considering', 'staying', 'give', 'chance', 'location', 'prime

## Embedding

In [ ]:
model = doc2vec.Doc2Vec(vector_size = 300, window = 15, min_count = 1, workers = 2, epochs=10)

model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=100)
model.save('d2v.model')

In [ ]:
doc2vec_model = model

In [ ]:
#len(doc2vec_model.docvecs)

In [ ]:
v = []

for i in range(len(df)):
    v.append(doc2vec_model.docvecs[i])

<ipython-input-37-737b6698745b>:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  v.append(doc2vec_model.docvecs[i])


In [ ]:
# X = v
# y = df['Real']

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Decision tree

In [ ]:
params_range_dt = {'criterion': hp.choice('criterion', ['gini', 'entropy']),
                   'max_depth': hp.quniform('max_depth', 2, 10, 1),
                   'splitter': hp.choice('splitter', ['best', 'random'])
}

In [ ]:
model_dt = DecisionTreeClassifier()

In [ ]:
def objective_dt(params_range):
    err = cross_val_score(model_dt, X_train, y_train, cv = 5, scoring='accuracy').mean()
 
    return {'loss': -err, 'status': STATUS_OK }

In [ ]:
trials = Trials()
best_params_dt = fmin(fn= objective_dt,
            space= params_range_dt,
            algo= tpe.suggest,
            max_evals = 50,
            trials= trials)

100%|██████████| 50/50 [08:48<00:00, 10.57s/trial, best loss: -0.7430523917995444]


In [ ]:
print('The best parameters are:', best_params_dt)

The best parameters are: {'criterion': 1, 'max_depth': 3.0, 'splitter': 1}


In [ ]:
model_dt_test = DecisionTreeClassifier(criterion=['gini', 'entropy'][best_params_dt['criterion']],
                                       max_depth=int(best_params_dt['max_depth']),
                                       splitter=['best', 'random'][best_params_dt['splitter']])
model_dt_test.fit(X_train, y_train)
y_pred_dt = model_dt_test.predict(X_test)

accuracy_dt = accuracy_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt, average='weighted')
print('Test accuracy: ', accuracy_dt, '\nTest F1: ', f1_dt)

Test accuracy:  0.8633879781420765 
Test F1:  0.8032744411999384


In [ ]:
# model_dt = DecisionTreeClassifier()
# acc_scores_dt = cross_val_score(model_dt, X, y, cv=10)
# f1_scores_dt = cross_val_score(model_dt, X, y, cv=10, scoring='f1')

In [ ]:
# print('Accuracy: ', acc_scores_dt.mean(), '\nF1: ', f1_scores_dt.mean())

### Random Forest

In [ ]:
params_range_rf = {'criterion': hp.choice('criterion', ['gini', 'entropy']),
                   'max_depth': hp.quniform('max_depth', 2, 10, 1),
                   'n_estimators': hp.quniform('n_estimators', 100, 1000, 1)
}

In [ ]:
model_rf = RandomForestClassifier()

In [ ]:
def objective_rf(params_range):
    err = cross_val_score(model_rf, X_train, y_train, cv = 5, scoring='accuracy').mean()
 
    return {'loss': -err, 'status': STATUS_OK }

In [ ]:
trials = Trials()
best_params_rf = fmin(fn= objective_rf,
            space= params_range_rf,
            algo= tpe.suggest,
            max_evals = 50,
            trials= trials)

100%|██████████| 50/50 [30:42<00:00, 36.84s/trial, best loss: -0.8676537585421412]


In [ ]:
print('The best parameters are:', best_params_rf)

The best parameters are: {'criterion': 1, 'max_depth': 9.0, 'n_estimators': 645.0}


In [ ]:
model_rf_test = RandomForestClassifier(criterion=['gini', 'entropy'][best_params_rf['criterion']],
                                       max_depth=int(best_params_rf['max_depth']),
                                       n_estimators=int(best_params_rf['n_estimators']))
model_rf_test.fit(X_train, y_train)
y_pred_rf = model_rf_test.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')
print('Test accuracy: ', accuracy_rf, '\nTest F1: ', f1_rf)

Test accuracy:  0.8654371584699454 
Test F1:  0.8030090661160166


### Support Vector Machine

In [ ]:
params_range_svm = {'kernel': hp.choice('kernel', ['RBF', 'linear', 'polynomial', 'sigmoid']),
                    'C': hp.uniform('C', 0.1, 1000),
                    'degree': hp.quniform('degree', 2, 10, 1),
                    'gamma': hp.choice('gamma', ['scale', 'auto'])
}

In [ ]:
model_svm = SVC()

In [ ]:
def objective_svm(params_range):
    err = cross_val_score(model_svm, X_train, y_train, cv = 5, scoring='accuracy').mean()
 
    return {'loss': -err, 'status': STATUS_OK }

In [ ]:
trials = Trials()
best_params_svm = fmin(fn= objective_svm,
            space= params_range_svm,
            algo= tpe.suggest,
            max_evals = 50,
            trials= trials)

100%|██████████| 50/50 [12:25<00:00, 14.92s/trial, best loss: -0.8676537585421412]


In [ ]:
print('The best parameters are:', best_params_svm)

The best parameters are: {'C': 607.7604274811972, 'degree': 7.0, 'gamma': 1, 'kernel': 0}


In [ ]:
model_svm_test = SVC(kernel=['rbf', 'linear', 'polynomial', 'sigmoid'][best_params_svm['kernel']],
                     C=best_params_svm['C'],
                     degree=int(best_params_svm['degree']),
                     gamma=['scale', 'auto'][best_params_svm['gamma']])
model_svm_test.fit(X_train, y_train)
y_pred_svm = model_svm_test.predict(X_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm, average='weighted')
print('Test accuracy: ', accuracy_svm, '\nTest F1: ', f1_svm)

Test accuracy:  0.7868852459016393 
Test F1:  0.7793498156788098


In [ ]:
# model_svm = SVC(kernel='rbf', C=21.832, degree=6, gamma='scale')
# acc_scores_svm = cross_val_score(model_svm, X, y, cv=10)
# f1_scores_svm = cross_val_score(model_svm, X, y, cv=10, scoring='f1')

### Extreme Gradient Boosting Trees

In [ ]:
l = []
for i in range(len(X_train)):
    l.append(list(X_train[i]))
l = np.reshape(l, (len(X_train), 300))

In [ ]:
params_range_xgb = {'max_depth': hp.quniform('max_depth', 2, 10, 1),
                    'n_estimators': hp.quniform('n_estimators', 100, 1000, 1),
                    'alpha': hp.uniform('alpha', 0.01, 1),
                    'gamma': hp.uniform('gamma', 0, 10)
}

In [ ]:
model_xgb = XGBClassifier()

In [ ]:
def objective_xgb(params_range):
    err = cross_val_score(model_xgb, l, y_train, cv = 5, scoring='accuracy').mean()
 
    return {'loss': -err, 'status': STATUS_OK }

In [ ]:
trials = Trials()
best_params_xgb = fmin(fn= objective_xgb,
            space= params_range_xgb,
            algo= tpe.suggest,
            max_evals = 25,
            trials= trials)

100%|██████████| 25/25 [46:26<00:00, 111.45s/trial, best loss: -0.8658314350797267]


In [ ]:
print('The best parameters are:', best_params_xgb)

The best parameters are: {'alpha': 0.4653157912592566, 'gamma': 0.5064562394773253, 'max_depth': 8.0, 'n_estimators': 189.0}


In [ ]:
l_test = []
for i in range(len(X_test)):
    l_test.append(list(X_test[i]))
l_test = np.reshape(l_test, (len(X_test), 300))

In [ ]:
model_xgb_test = XGBClassifier(max_depth=int(best_params_xgb['max_depth']),
                               n_estimators=int(best_params_xgb['n_estimators']),
                               gamma=best_params_xgb['gamma'],
                               alpha=best_params_xgb['alpha'])
model_xgb_test.fit(l, y_train)
y_pred_xgb = model_xgb_test.predict(l_test)

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')
print('Test accuracy: ', accuracy_xgb, '\nTest F1: ', f1_xgb)

Test accuracy:  0.8620218579234973 
Test F1:  0.8038258168313869


In [ ]:
# model_xgb = XGBClassifier(n_estimators=421, max_depth=3, gamma=9.08, alpha=0.035)
# acc_scores_xgb = cross_val_score(model_xgb, l, y, cv=10)
# f1_scores_xgb = cross_val_score(model_xgb, l, y, cv=10, scoring='f1')

### Multilayer perceptron

In [ ]:
hls = []
for i in range(20):
    rt = np.random.randint(1,10,size=2)
    hls.append(tuple(rt*5))

    rt = np.random.randint(1,10,size=3)
    hls.append(tuple(rt*5))

    rt = np.random.randint(1,10,size=4)
    hls.append(tuple(rt*5))

    rt = np.random.randint(1,10,size=5)
    hls.append(tuple(rt*5))

In [ ]:
params_range_mlp = {'hidden_layer_sizes': hp.choice('hidden_layer_sizes', hls),
                    'max_iter': hp.quniform('max_iter', 400, 1200, 100),
                    'activation': hp.choice('activation', ['relu', 'identity', 'logistic', 'tanh'])
}

In [ ]:
model_mlp = MLPClassifier()

In [ ]:
def objective_mlp(params_range):
    err = cross_val_score(model_mlp, X_train, y_train, cv = 5, scoring='accuracy').mean()
 
    return {'loss': -err, 'status': STATUS_OK }

In [ ]:
trials = Trials()
best_params_mlp = fmin(fn= objective_mlp,
            space= params_range_mlp,
            algo= tpe.suggest,
            max_evals = 25,
            trials= trials)

100%|██████████| 25/25 [21:08<00:00, 50.72s/trial, best loss: -0.8389521640091117]


In [ ]:
print('The best parameters are:', best_params_mlp)

The best parameters are: {'activation': 0, 'hidden_layer_sizes': 72, 'max_iter': 900.0}


In [ ]:
model_mlp_test = MLPClassifier(activation=['relu', 'identity', 'logistic', 'tanh'][best_params_mlp['activation']],
                               max_iter=int(best_params_mlp['max_iter']),
                               hidden_layer_sizes=hls[best_params_mlp['hidden_layer_sizes']])
model_mlp_test.fit(X_train, y_train)
y_pred_mlp = model_mlp_test.predict(X_test)

accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
f1_mlp = f1_score(y_test, y_pred_mlp, average='weighted')
print('Test accuracy: ', accuracy_mlp, '\nTest F1: ', f1_mlp)

Test accuracy:  0.7773224043715847 
Test F1:  0.7768421559035777


In [ ]:
print('Accuracy DT: ', accuracy_dt, '\nF1 DT: ', f1_dt)
print('Accuracy RF: ', accuracy_rf, '\nF1 RF: ', f1_rf)
print('Accuracy SVM: ', accuracy_svm, '\nF1 SVM: ', f1_svm)
print('Accuracy XGB: ', accuracy_xgb, '\nF1 XGB: ', f1_xgb)
print('Accuracy MLP: ', accuracy_mlp, '\nF1 MLP: ', f1_mlp)

Accuracy DT:  0.8633879781420765 
F1 DT:  0.8032744411999384
Accuracy RF:  0.8654371584699454 
F1 RF:  0.8030090661160166
Accuracy SVM:  0.7868852459016393 
F1 SVM:  0.7793498156788098
Accuracy XGB:  0.8620218579234973 
F1 XGB:  0.8038258168313869
Accuracy MLP:  0.7773224043715847 
F1 MLP:  0.7768421559035777


In [ ]:
columns = ['Average word length', 'Pausality', 'Average NP length', 'Average sentence length', 'Number of clauses', 'Number of words', 'Number of verbs', 
           'Number of adjectives', 'Number of passive voice', 'Emotiveness', 'Content diversity', 'Redundancy', 'Lexical diversity', 'Number of modal verbs', 'Number of typos']

In [ ]:
df_features = pd.read_csv('/content/drive/MyDrive/Fake reviews detection/chi_hotel_features.csv')

## Experimenting

In [ ]:
#df_f1 = df_features[['Average word length', 'Content diversity', 'Emotiveness']]
df_f1 = df_features

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(df_f1, df['Real'], test_size = 0.33, random_state=0)

### Decision tree + features

In [ ]:
params_range_dt = {'criterion': hp.choice('criterion', ['gini', 'entropy']),
                   'max_depth': hp.quniform('max_depth', 2, 10, 1),
                   'splitter': hp.choice('splitter', ['best', 'random'])
}

In [ ]:
model_dt_f = DecisionTreeClassifier()

In [ ]:
def objective_dt_f(params_range):
    err = cross_val_score(model_dt_f, X_train1, y_train1, cv = 5, scoring='accuracy').mean()
 
    return {'loss': -err, 'status': STATUS_OK }

In [ ]:
trials = Trials()
best_params_dt_f = fmin(fn= objective_dt_f,
            space= params_range_dt,
            algo= tpe.suggest,
            max_evals = 50,
            trials= trials)

100%|██████████| 50/50 [00:14<00:00,  3.35trial/s, best loss: -0.7580407513323799]


In [ ]:
print('The best parameters are:', best_params_dt_f)

The best parameters are: {'criterion': 1, 'max_depth': 8.0, 'splitter': 0}


In [ ]:
model_dt_test_f = DecisionTreeClassifier(criterion=['gini', 'entropy'][best_params_dt_f['criterion']],
                                       max_depth=int(best_params_dt_f['max_depth']),
                                       splitter=['best', 'random'][best_params_dt_f['splitter']])
model_dt_test_f.fit(X_train1, y_train1)
y_pred_dt_f = model_dt_test_f.predict(X_test1)

accuracy_dt_f = accuracy_score(y_test1, y_pred_dt_f)
f1_dt_f = f1_score(y_test1, y_pred_dt_f, average='weighted')
print('Test accuracy: ', accuracy_dt_f, '\nTest F1: ', f1_dt_f)

Test accuracy:  0.8524844720496895 
Test F1:  0.8095973146140371


In [ ]:
# model_dt_f = DecisionTreeClassifier(criterion='gini', max_depth=3, splitter='best')
# acc_scores_dt_f = cross_val_score(model_dt_f, X_train1, y_train1, cv=10)
# f1_scores_dt_f = cross_val_score(model_dt_f, X_train1, y_train1, cv=10, scoring='f1')

In [ ]:
# cv = StratifiedKFold(n_splits=10)
# for (train, test), i in zip(cv.split(X_train1, y_train1), range(10)):
#     model_dt_f.fit(X_train1, y_train1)

### Random Forest + features

In [ ]:
params_range_rf = {'criterion': hp.choice('criterion', ['gini', 'entropy']),
                   'max_depth': hp.quniform('max_depth', 2, 10, 1),
                   'n_estimators': hp.quniform('n_estimators', 100, 1000, 1)
}

In [ ]:
model_rf_f = RandomForestClassifier()

In [ ]:
def objective_rf_f(params_range):
    err = cross_val_score(model_rf_f, X_train1, y_train1, cv = 5, scoring='accuracy').mean()
 
    return {'loss': -err, 'status': STATUS_OK }

In [ ]:
trials = Trials()
best_params_rf_f = fmin(fn= objective_rf_f,
            space= params_range_rf,
            algo= tpe.suggest,
            max_evals = 50,
            trials= trials)

100%|██████████| 50/50 [04:09<00:00,  4.99s/trial, best loss: -0.8671594306512415]


In [ ]:
print('The best parameters are:', best_params_rf_f)

The best parameters are: {'criterion': 0, 'max_depth': 8.0, 'n_estimators': 117.0}


In [ ]:
model_rf_test_f = RandomForestClassifier(criterion=['gini', 'entropy'][best_params_rf_f['criterion']],
                                       max_depth=int(best_params_rf_f['max_depth']),
                                       n_estimators=int(best_params_rf_f['n_estimators']))
model_rf_test_f.fit(X_train1, y_train1)
y_pred_rf_f = model_rf_test_f.predict(X_test1)

accuracy_rf_f = accuracy_score(y_test1, y_pred_rf_f)
f1_rf_f = f1_score(y_test1, y_pred_rf_f, average='weighted')
print('Test accuracy: ', accuracy_rf_f, '\nTest F1: ', f1_rf_f)

Test accuracy:  0.8664596273291926 
Test F1:  0.8044666756234434


In [ ]:
# model_rf_f = RandomForestClassifier(n_estimators=827, criterion='entropy', max_depth=4)
# acc_scores_rf_f = cross_val_score(model_rf_f, X_train1, y_train1, cv=10)
# f1_scores_rf_f = cross_val_score(model_rf_f, X_train1, y_train1, cv=10, scoring='f1')

In [ ]:
# cv = StratifiedKFold(n_splits=10)
# for (train, test), i in zip(cv.split(X_train1, y_train1), range(10)):
#     model_rf_f.fit(X_train1, y_train1)

### Support Vector Machine + features

In [ ]:
params_range_svm = {'kernel': hp.choice('kernel', ['rbf', 'linear', 'polynomial', 'sigmoid']),
                    'C': hp.uniform('C', 0.1, 1000),
                    'degree': hp.quniform('degree', 2, 10, 1),
                    'gamma': hp.choice('gamma', ['scale', 'auto'])
}

In [ ]:
model_svm_f = SVC()

In [ ]:
def objective_svm_f(params_range):
    err = cross_val_score(model_svm_f, X_train1, y_train1, cv = 5, scoring='accuracy').mean()
 
    return {'loss': -err, 'status': STATUS_OK }

In [ ]:
trials = Trials()
best_params_svm_f = fmin(fn= objective_svm_f,
            space= params_range_svm,
            algo= tpe.suggest,
            max_evals = 50,
            trials= trials)

100%|██████████| 50/50 [01:18<00:00,  1.57s/trial, best loss: -0.8674145326920577]


In [ ]:
print('The best parameters are:', best_params_svm_f)

The best parameters are: {'C': 645.3771328311205, 'degree': 4.0, 'gamma': 1, 'kernel': 0}


In [ ]:
model_svm_test_f = SVC(kernel=['rbf', 'linear', 'polynomial', 'sigmoid'][best_params_svm_f['kernel']],
                     C=best_params_svm_f['C'],
                     degree=int(best_params_svm_f['degree']),
                     gamma=['scale', 'auto'][best_params_svm_f['gamma']])
model_svm_test_f.fit(X_train1, y_train1)
y_pred_svm_f= model_svm_test_f.predict(X_test1)

accuracy_svm_f = accuracy_score(y_test1, y_pred_svm_f)
f1_svm_f = f1_score(y_test1, y_pred_svm_f, average='weighted')
print('Test accuracy: ', accuracy_svm_f, '\nTest F1: ', f1_svm_f)

Test accuracy:  0.8255693581780539 
Test F1:  0.7963010063571581


In [ ]:
# model_svm_f = SVC(kernel='rbf', C=21.832, degree=6, gamma='scale')
# acc_scores_svm_f = cross_val_score(model_svm_f, X_train1, y_train1, cv=10)
# f1_scores_svm_f= cross_val_score(model_svm_f, X_train1, y_train1, cv=10, scoring='f1')

In [ ]:
# cv = StratifiedKFold(n_splits=10)
# for (train, test), i in zip(cv.split(X_train1, y_train1), range(10)):
#     model_svm_f.fit(X_train1, y_train1)

### Extreme Gradient Boosting Trees

In [ ]:
params_range_xgb = {'max_depth': hp.quniform('max_depth', 2, 10, 1),
                    'n_estimators': hp.quniform('n_estimators', 100, 1000, 1),
                    'alpha': hp.uniform('alpha', 0.01, 1),
                    'gamma': hp.uniform('gamma', 0, 10)
}

In [ ]:
model_xgb_f = XGBClassifier()

In [ ]:
def objective_xgb_f(params_range):
    err = cross_val_score(model_xgb_f, X_train1, y_train1, cv = 5, scoring='accuracy').mean()
 
    return {'loss': -err, 'status': STATUS_OK }

In [ ]:
trials = Trials()
best_params_xgb_f = fmin(fn= objective_xgb_f,
            space= params_range_xgb,
            algo= tpe.suggest,
            max_evals = 25,
            trials= trials)

100%|██████████| 25/25 [01:36<00:00,  3.84s/trial, best loss: -0.8521171844533992]


In [ ]:
print('The best parameters are:', best_params_xgb_f)

The best parameters are: {'alpha': 0.6722998212659704, 'gamma': 3.9502959793516057, 'max_depth': 6.0, 'n_estimators': 721.0}


In [ ]:
model_xgb_test_f = XGBClassifier(max_depth=int(best_params_xgb_f['max_depth']),
                               n_estimators=int(best_params_xgb_f['n_estimators']),
                               gamma=best_params_xgb_f['gamma'],
                               alpha=best_params_xgb_f['alpha'])
model_xgb_test_f.fit(X_train1, y_train1)
y_pred_xgb_f = model_xgb_test_f.predict(X_test1)

accuracy_xgb_f = accuracy_score(y_test1, y_pred_xgb_f)
f1_xgb_f = f1_score(y_test1, y_pred_xgb_f, average='weighted')
print('Test accuracy: ', accuracy_xgb_f, '\nTest F1: ', f1_xgb_f)

Test accuracy:  0.8659420289855072 
Test F1:  0.8071289057825515


In [ ]:
# model_xgb_f = XGBClassifier(n_estimators=421, max_depth=3, gamma=9.08, alpha=0.035)
# acc_scores_xgb_f = cross_val_score(model_xgb_f, X_train1, y_train1, cv=10)
# f1_scores_xgb_f = cross_val_score(model_xgb_f, X_train1, y_train1, cv=10, scoring='f1')

In [ ]:
# cv = StratifiedKFold(n_splits=10)
# for (train, test), i in zip(cv.split(X_train1, y_train1), range(10)):
#     model_xgb_f.fit(X_train1, y_train1)

### Multilayer perceptron + features

In [ ]:
hls = []
for i in range(20):
    rt = np.random.randint(1,10,size=2)
    hls.append(tuple(rt*5))

    rt = np.random.randint(1,10,size=3)
    hls.append(tuple(rt*5))

    rt = np.random.randint(1,10,size=4)
    hls.append(tuple(rt*5))

    rt = np.random.randint(1,10,size=5)
    hls.append(tuple(rt*5))

In [ ]:
params_range_mlp = {'hidden_layer_sizes': hp.choice('hidden_layer_sizes', hls),
                    'max_iter': hp.quniform('max_iter', 400, 1200, 100),
                    'activation': hp.choice('activation', ['relu', 'identity', 'logistic', 'tanh'])
}

In [ ]:
model_mlp_f = MLPClassifier()

In [ ]:
def objective_mlp_f(params_range):
    err = cross_val_score(model_mlp_f, X_train1, y_train1, cv = 5, scoring='accuracy').mean()
 
    return {'loss': -err, 'status': STATUS_OK }

In [ ]:
trials = Trials()
best_params_mlp_f = fmin(fn= objective_mlp_f,
            space= params_range_mlp,
            algo= tpe.suggest,
            max_evals = 25,
            trials= trials)

100%|██████████| 25/25 [02:30<00:00,  6.02s/trial, best loss: -0.8674145326920577]


In [ ]:
print('The best parameters are:', best_params_mlp_f)

The best parameters are: {'activation': 0, 'hidden_layer_sizes': 21, 'max_iter': 900.0}


In [ ]:
model_mlp_test_f = MLPClassifier(activation=['relu', 'identity', 'logistic', 'tanh'][best_params_mlp_f['activation']],
                               max_iter=int(best_params_mlp_f['max_iter']),
                               hidden_layer_sizes=hls[best_params_mlp_f['hidden_layer_sizes']])
model_mlp_test_f.fit(X_train1, y_train1)
y_pred_mlp_f = model_mlp_test_f.predict(X_test1)

accuracy_mlp_f = accuracy_score(y_test1, y_pred_mlp_f)
f1_mlp_f = f1_score(y_test1, y_pred_mlp_f, average='weighted')
print('Test accuracy: ', accuracy_mlp_f, '\nTest F1: ', f1_mlp_f)

Test accuracy:  0.8664596273291926 
Test F1:  0.8044666756234434


In [ ]:
print('Accuracy DT: ', accuracy_dt_f, '\nF1 DT: ', f1_dt_f)
print('Accuracy RF: ', accuracy_rf_f, '\nF1 RF: ', f1_rf_f)
print('Accuracy SVM: ', accuracy_svm_f, '\nF1 SVM: ', f1_svm_f)
print('Accuracy XGB: ', accuracy_xgb_f, '\nF1 XGB: ', f1_xgb_f)
print('Accuracy MLP: ', accuracy_mlp_f, '\nF1 MLP: ', f1_mlp_f)

Accuracy DT:  0.8524844720496895 
F1 DT:  0.8095973146140371
Accuracy RF:  0.8664596273291926 
F1 RF:  0.8044666756234434
Accuracy SVM:  0.8255693581780539 
F1 SVM:  0.7963010063571581
Accuracy XGB:  0.8659420289855072 
F1 XGB:  0.8071289057825515
Accuracy MLP:  0.8664596273291926 
F1 MLP:  0.8044666756234434
